In [119]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf

In [120]:
def list_classes_and_count(dataset_dir):
    classes = set()
    class_count = 0
    for root, dirs, files in os.walk(dataset_dir):
        for dir in dirs:
            classes.add(dir)
            class_count += 1
    return classes, class_count


# Directory paths for the two datasets
dataset_dir = "New Plant Diseases Dataset"

# List classes and count in each dataset
dataset_classes, dataset_classes = list_classes_and_count(dataset_dir + "/train")

# Display unique classes and the number of classes in each dataset
print("Classes in New Plant Diseases Dataset:")
print(dataset_classes)
print(f"Number of classes: {dataset_classes}")

Classes in New Plant Diseases Dataset:
38
Number of classes: 38


In [121]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Image data generators for both datasets with augmentation
datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2,
)

In [122]:
BATCH_SIZE = 32
IMG_SIZE = (64, 64)


train_dataset = datagen.flow_from_directory(
    dataset_dir + "/train",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    subset="training",
)

valid_dataset = datagen.flow_from_directory(
    dataset_dir + "/valid",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    subset="validation",
)

Found 56251 images belonging to 38 classes.
Found 3503 images belonging to 38 classes.


In [123]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Concatenate
from tensorflow.keras.models import Model

# Load ResNet50 model without including the top fully-connected layers
IMG_SHAPE = IMG_SIZE + (3,)
base_model = ResNet50(input_shape=IMG_SHAPE, weights="imagenet", include_top=False)

base_model.trainable = True

# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

# Fine-tune from this layer onwards
fine_tune_at = 100

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

Number of layers in the base model:  175


In [124]:
# Let's take a look at the base model architecture
base_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_12 (InputLayer)       [(None, 64, 64, 3)]          0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 70, 70, 3)            0         ['input_12[0][0]']            
                                                                                                  
 conv1_conv (Conv2D)         (None, 32, 32, 64)           9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 32, 32, 64)           256       ['conv1_conv[0][0]']          
 on)                                                                                       

In [125]:
# Add custom top layers for classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
# x = Dense(1024, activation="relu")(x)
x = tf.keras.layers.Dropout(0.2)(x)
predictions = Dense(38, activation="softmax")(x)

# Create the model
model = Model(inputs=base_model.input, outputs=predictions)

In [126]:
base_learning_rate = 0.001

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.RMSprop(learning_rate=base_learning_rate / 10),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)

In [127]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    monitor="val_accuracy", patience=2, restore_best_weights=True, verbose=1
)

In [128]:
# Train the model
history = model.fit(
    train_dataset,
    epochs=100,
    validation_data=valid_dataset,
    callbacks=[early_stopping],
)

Epoch 1/100
  42/1758 [..............................] - ETA: 24:49 - loss: 3.6727 - accuracy: 0.0588